# Gomoku AI Analysis Dashboard

Exploratory Data Analysis (EDA) of self-play evaluation results.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os

# Settings
sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)

print("Libraries loaded.")

## 1. Load Data

In [ ]:
# Load latest results
data_files = glob("../data/results/self_play/aggregated/results_*.csv")
if data_files:
    latest_file = max(data_files, key=os.path.getctime)
    df = pd.read_csv(latest_file)
    print(f"Loaded: {latest_file}")
    print(f"Total records: {len(df)}")
else:
    print("No data files found.")
    df = None

In [ ]:
if df is not None: display(df.head())

In [ ]:
if df is not None: df.info()

## 2. Win Rate Analysis

In [ ]:
if df is not None:
    # Winner Distribution
    winner_counts = df['winner'].value_counts()
    print("Winner Distribution:")
    print(winner_counts)

    plt.figure(figsize=(8, 6))
    winner_counts.plot(kind='bar', color=['green', 'red', 'gray'])
    plt.title('Winner Distribution', fontsize=16)
    plt.xlabel('Winner')
    plt.ylabel('Count')
    plt.show()

In [ ]:
if df is not None:
    # Calculate Win Rates
    algorithms = sorted(set(df['player1'].unique()) | set(df['player2'].unique()))
    win_stats = []

    for algo in algorithms:
        # As P1
        as_p1 = df[df['player1'] == algo]
        p1_wins = (as_p1['winner'] == 'player1').sum()
        
        # As P2
        as_p2 = df[df['player2'] == algo]
        p2_wins = (as_p2['winner'] == 'player2').sum()
        
        total = len(as_p1) + len(as_p2)
        wins = p1_wins + p2_wins
        
        win_stats.append({
            'algorithm': algo,
            'games': total,
            'wins': wins,
            'rate': wins / total if total > 0 else 0
        })

    win_df = pd.DataFrame(win_stats).sort_values('rate', ascending=False)
    display(win_df)
    
    # Visualize
    plt.figure(figsize=(10, 6))
    colors = sns.color_palette('RdYlGn', len(win_df))
    plt.barh(win_df['algorithm'], win_df['rate'], color=colors)
    plt.xlabel('Win Rate')
    plt.title('Algorithm Win Rates')
    plt.xlim(0, 1)
    plt.show()

## 3. Game Length Analysis

In [ ]:
if df is not None:
    print(f"Avg Moves: {df['total_moves'].mean():.1f}")
    print(f"Max Moves: {df['total_moves'].max()}")

    plt.figure(figsize=(10, 6))
    plt.hist(df['total_moves'], bins=30, edgecolor='black', alpha=0.7)
    plt.axvline(df['total_moves'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Moves')
    plt.ylabel('Frequency')
    plt.title('Game Length Distribution')
    plt.legend()
    plt.show()

## 4. Response Time Analysis

In [ ]:
if df is not None:
    time_stats = []
    for algo in algorithms:
        t1 = df[df['player1'] == algo]['player1_avg_time']
        t2 = df[df['player2'] == algo]['player2_avg_time']
        all_times = pd.concat([t1, t2])
        
        time_stats.append({
            'algorithm': algo,
            'avg_time': all_times.mean()
        })
    
    time_df = pd.DataFrame(time_stats).sort_values('avg_time')
    
    plt.figure(figsize=(10, 6))
    plt.bar(time_df['algorithm'], time_df['avg_time'])
    plt.ylabel('Time (s)')
    plt.title('Average Response Time')
    plt.xticks(rotation=45)
    plt.show()